# Profile stream_bulk_split

A simple notebook to profile the function `stream_bulk_split` from obsplus.waveforms.utils

In [ ]:
import timeit
from functools import reduce
from operator import add

import obsplus
import obspy
import pandas as pd
from obsplus.waveforms.utils import stream_bulk_split

In [ ]:
# define params to profile
# num_traces = [3, 6, 12, 30, 60, 90]
# num_bulk = [3, 6, 12, 30]


num_traces = [6, 18, 24]
num_bulk = [6, 12, 24]

In [ ]:
# get stream(s)
stream_dict = {}
for stream_num in num_traces:
    st = reduce(add, [obspy.read() for _ in range(stream_num // 3)])
    stream_dict[stream_num] = st

In [ ]:
st_original = obspy.read()
# get bulk dict
bulk_dict = {}
for bulk_num in num_bulk:
    bulk = []
    for bnum in range(bulk_num // 3):
        sub_bulk = []
        for tr in st_original:
            t1 = tr.stats.starttime + 1
            t2 = tr.stats.endtime + 1
            sub_bulk.append(tr.id.split('.') + [t1, t2])
        bulk.extend(sub_bulk)
    bulk_dict[bulk_num] = bulk

In [ ]:
df = pd.DataFrame(columns=list(stream_dict), index=list(bulk_dict))
df.columns.name = 'trace_count'
df.index.name = 'bulk_count'

for tr_num, st in stream_dict.items():
    for bulk_num, bulk in bulk_dict.items():
        print(f'bulk: {bulk_num}, tr_num: {tr_num}')
        assert len(st) == tr_num
        assert len(bulk) == bulk_num
        
        out = %timeit -o stream_bulk_split(st, bulk)
        print('')
        df.loc[bulk_num, tr_num] = out.average
        
#         %timeit stream_bulk_split(st, bulk)
        


In [ ]:
df.head()

In [ ]:
# %load_ext snakeviz

In [ ]:
# %%snakeviz
# stream_bulk_split(st, bulk)